In [2]:
"""
@author: Dagang Yang


"""


'\n@author: Dagang Yang\n\n\n'

In [3]:
"""
Parameters:
T:     Maturity (years)
n:     # Option periods
N:     # Future periods
S:     Stock price at t=0
r:     Annual risk-free rate
c:     Divident yield
sigma: Annualized volatility
K:     Strike price
cp:    call = 1; put = -1
"""

'\nParameters:\nT:     Maturity (years)\nn:     # Option periods\nN:     # Future periods\nS:     Stock price at t=0\nr:     Annual risk-free rate\nc:     Divident yield\nsigma: Annualized volatility\nK:     Strike price\ncp:    call = 1; put = -1\n'

In [190]:
import numpy as np
from math import exp, sqrt
np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)}) #print 2 decimals only


In [91]:
def Parameters(sigma, T, n, r, c):
    u = exp(sigma * sqrt(T/n))
    d = 1/u
    q = (exp((r-c) * T / n) - d)/(u - d)
    R = exp(r*T/n)
    
    return u, d, q, R

In [185]:
def Tree_StockPrice(T, n, r, c, sigma, K, S):
    u, d, q, R = Parameters(sigma, T, n, r, c)
    Stock_Lattice = np.zeros((n+1, n+1))
    Stock_Lattice[0, 0] = S      
    for j in range(1, n+1):
        Stock_Lattice[0, j] = Stock_Lattice[0, j-1] * u        # First row, stock price goes up all the time
        for i in range(1, j+1):
            Stock_Lattice[i, j] = Stock_Lattice[i-1, j-1] * d  # Following rows, stock price goes down from previous state
    #print("First five states of the stock price are:")
    #print(Stock_Lattice[:5,:5])
    return Stock_Lattice

In [191]:
def Stock_Option_EU(T,n,S,r,c,sigma,K,cp):
    '''
    To calculate the fair value of an European option
    '''
    u, d, q, R = Parameters(sigma, T, n, r, c)
    #print("Parameters:\nu = %.4f\nd = %.4f\nq = %.4f\n1-q = %.4f\nR = %.4f\n" %(u,d,q,1-q,R))
    
    Stock_Lattice = Tree_StockPrice(T,n,r,c,sigma,K,S)
    Option_Lattice = np.zeros((n+1, n+1))

    # Last column of option value
    for i in range(n+1):
        Option_Lattice[i, n] = max(0, cp*(Stock_Lattice[i, n]-K))

    # Compute reversely
    for j in range(n-1, -1, -1):
        for i in range(j+1):
            Option_Lattice[i, j] = (q*Option_Lattice[i, j+1]+ (1-q)*Option_Lattice[i+1, j+1])/R
    '''
    if cp == 1:        
        print("The fair price of the call option is %.2f" %Option_Lattice[0,0])
    else:
        print("The fair price of the put option is %.2f" %Option_Lattice[0,0])
    '''
    return Option_Lattice[0,0]
    

In [192]:
Stock_Option_EU(0.25,15,100,0.02,0.01,0.3,110,1)

np.float64(2.6040771329665637)

In [256]:
def Put_Call_Parity_Check(T,n,S,r,c,sigma,K):
    # Check put call parity for european options only
    u, d, q, R = Parameters(sigma, T, n, r, c)
    
    call = Stock_Option_EU(T,n,S,r,c,sigma,K,1)
    put = Stock_Option_EU(T,n,S,r,c,sigma,K,-1)

    print("\nCheck Put Call Parity:\nP + S*e^(-cT) = C + K*d(0,T)")
    print("%.5f + %.5f*%.5f = %.5f + %.5f/%.5f" %(put, S, exp(-c*T), call, K, exp(r*T)))
    print("lhs = %.5f" %(put + S*exp(-c*T)))
    print("rhs = %.5f" %(call + K/exp(r*T)))

    # Compare 10 decimal places
    print((put + S*exp(-c*T)).round(10)==(call + K/exp(r*T)).round(10)) 

In [ ]:
# Check 2 with dividend and 2 without 
Put_Call_Parity_Check(0.01, 1, 100, 0.02, 0.01, 0.5, 105)
Put_Call_Parity_Check(0.25, 15, 100, 0.02, 0.01, 0.3, 110)
Put_Call_Parity_Check(0.25, 15, 100, 0.02, 0, 0.3, 110)
Put_Call_Parity_Check(0.5, 25, 200, 0.05, 0, 0.3, 250)

In [247]:
def Stock_Option_AM(T,n,S,r,c,sigma,K,cp=-1):
    '''
    Put Option only - For call options American = European
    Returns fair price of and the number of first stage that possibly exercise the option
    If the option is not considered to be exercised all the time, return -1    
    '''
    u, d, q, R = Parameters(sigma, T, n, r, c)
    Stock_Lattice = Tree_StockPrice(T,n,r,c,sigma,K,S)

    # The possible stages the option could be considered to exercise
    Option_Exe = [-1]

    Option_Lattice = np.zeros((n+1, n+1))
    # Last column of option value
    for i in range(n+1):
        Option_Lattice[i, n] = max(0, cp*(Stock_Lattice[i, n]-K))
        # Check to exercise or not
        if Option_Lattice[i, n]:
            Option_Exe.append(n)

    # Compute option value reversely
    for j in range(n-1, -1, -1):
        for i in range(j+1):
            Option_Lattice[i, j] = max((q*Option_Lattice[i, j+1]+ (1-q)*Option_Lattice[i+1, j+1])/R, cp*(Stock_Lattice[i,j] - K))

            # Check to exercise or not
            if Option_Lattice[i, j] == cp*(Stock_Lattice[i,j] - K):
                Option_Exe.append(i)
    #print(Option_Lattice)
    return (Option_Lattice[0,0], Option_Exe[-1])

In [248]:
Stock_Option_AM(0.25, 15, 100, 0.02, 0.01, 0.3, 150, -1)

(np.float64(50.0), 0)